# Slice and Interpolate Image Data

In [ ]:
import pathlib 
from collections import defaultdict

import h5py
import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm,Normalize
import matplotlib.cm as cm

from bluesky_tutorial_utils import nexus, fetch

In [ ]:
fetch.rsoxs_simulation_data()

## Working with a Single Image xArray

In [ ]:
fname = './rsoxs_simulation_data/512-512-128-5.0-40.0-00285-0360.nxs'
da_img = nexus.read_singleimg_nxs(fname)
da_img_chi = nexus.read_singleimg_nxs(fname,sasdata='unwrap')
da_img

### Basic Slicing and Plotting 

First just a simple image plot on a log scale

In [ ]:
da_img.plot(norm=LogNorm(1e-9,1),aspect=1.2,size=5)

Comparison of nearest-pixel versus interpolated selection for a **horizontal line cut**

In [ ]:
da_img.sel(Qy=0,method='nearest').plot(yscale='log',xscale='log',label='nearest')
da_img.interp(Qy=0).plot(yscale='log',xscale='log',label='interp')
plt.legend()

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(16,4))
da_img_chi.plot(norm=LogNorm(1e-9,1),ax=axes[0])
da_img_chi.sel(Q=0.25,method='nearest').plot(ax=axes[1])
da_img_chi.sel(Q=0.35,method='nearest').plot(ax=axes[1],yscale='log')

(da_img_chi.sel(Q=0.35,method='nearest')
          .coarsen({'chi':5})
          .mean()
          .plot(ax=axes[1],yscale='log'))

da_img_chi.mean('chi').plot(ax=axes[2],xscale='log',yscale='log',label='Full Azimuthal')
da_img_chi.sel(chi=0,method='nearest').plot(ax=axes[2],xscale='log',yscale='log',label='Single χ')
da_img_chi.sel(chi=np.arange(-10,10,0.1),method='nearest').mean('chi').plot(ax=axes[2],xscale='log',yscale='log',label='Sector Average')
axes[2].legend()
plt.tight_layout()

### Remeshing

In [ ]:
qx = np.linspace(-0.5,0.5,512)
qy = np.linspace(-0.5,0.5,512)
da_img_remesh = da_img.interp(Qx=qx,Qy=qy)
da_img_remesh

In [ ]:
da_img_remesh.plot(norm=LogNorm(1e-9,1),aspect=1.2,size=5)

## Working with Multiple xArrays: Gathering Data

### Build Index Table (Table of Contents)

In [ ]:
def build_pandas_index(nxs_path):
    nxs_path = pathlib.Path(nxs_path)
    nxs_files = list(nxs_path.glob('*nxs'))
    
    #progress = ipywidgets.IntProgress(0,0,len(nxs_files))
    #display(progress)
    
    index_table = []
    for i,nxs_file in enumerate(nxs_files):
        #progress.value = i
        with h5py.File(nxs_file,'r') as nxs:
            notes = nxs[u'entry/instrument/simulation_engine/notes']
            config =  {k:v[()] for k,v in notes.items()}
            config['nxs'] = nxs_file
            index_table.append(config)
    return pd.DataFrame(index_table)

In [ ]:
toc = build_pandas_index('./rsoxs_simulation_data//')

In [ ]:
toc

In [ ]:
toc.describe().loc[['count','min','max']]

### Select subset of data From Index

In [ ]:
sdf = toc.query('Radius==40.0 & EndAngle==360.0 & PhysSize==5 & NumX==512')
sdf = sdf.sort_values('Energy')
sdf.describe().loc[['count','min','max']]

### Gather Data

In [ ]:
def gather(df):
    coords = defaultdict(list)
    data_arrays = []
    for row_index,row in df.iterrows():
        da_img = nexus.read_singleimg_nxs(row['nxs'])
        data_arrays.append(da_img)
        
        for col_index,value in row.iteritems():
            if col_index=='nxs':
                continue
            coords[col_index].append(value)
    return data_arrays,coords

In [ ]:
data_arrays,coords = gather(sdf)

In [ ]:
data_arrays

In [ ]:
coords

In [ ]:
data_arrays[1].plot(norm=LogNorm(1e-9,1),aspect=1.2,size=5)

## Multiple xArrays: simple xr.concat

In [ ]:
sdf = toc.query('Radius==40.0 & EndAngle==360.0 & PhysSize==5 & NumX==512')
sdf = sdf.sort_values('Energy')
display(sdf.describe().loc[['count','min','max']])

data_arrays,coords = gather(sdf)

da = xr.concat(data_arrays,dim='Energy')
da

In [ ]:
da = da.assign_coords(Energy=sdf.Energy.values)
da

In [ ]:
da.sel(Qy=0,method='nearest').plot(norm=LogNorm(1e-9,1),yscale='log')

In [ ]:
da.plot(col='Energy',col_wrap=3,norm=LogNorm(1e-9,1))

## Building xArrays: Multi-Index

In [ ]:
sdf = toc.query('EndAngle==360.0 & PhysSize==5 & NumX==512')
sdf = sdf.sort_values(['Energy','Radius'])
display(sdf.describe().loc[['count','min','max']])

In [ ]:
data_arrays,coords = gather(sdf)

da = xr.concat(data_arrays,dim=['Energy','Radius'])

hmmm...that didn't work...


Let's try a multi-index

In [ ]:
keys =  ['Energy','Radius']
tuples = [(i,j) for i,j in sdf[keys].values]
index = pd.MultiIndex.from_tuples(tuples,names=keys)
index.name = 'system'
da = xr.concat(data_arrays,dim=index)
da

In [ ]:
da.sel(Energy=285.,method='nearest')

In [ ]:
da.sel(Energy=285.,Qy=0,method='nearest').plot.line(x='Qx',yscale='log',xscale='log')#(norm=LogNorm(1e-9,1))

In [ ]:
da.sel(Energy=285.,method='nearest').plot(col='Radius',col_wrap=3,norm=LogNorm(1e-9,1))

## Extra: Multidimensional Coordinates

First let's look at our initial 2D array

In [ ]:
da_img

First we make a $Q=\sqrt{Q_{x}^2 + Q_{y}^2}$ array

In [ ]:
Q = np.sqrt(da_img.Qx*da_img.Qx + da_img.Qy*da_img.Qy)
Q.name = 'Q'
Q.plot()
Q

Then we make a $\chi= \tan^{-1}\left(\frac{Q_{y}}{Q_{x}}\right)$ array

In [ ]:
CHI = np.arctan2(da_img.Qy,da_img.Qx)*180/np.pi
CHI.name = 'χ'
CHI.plot()
CHI

finally we add the new coordinates to the image dataarray

In [ ]:
da = da_img.assign_coords(Q=Q,CHI=CHI)
da

we can use these multidimensional coordinates to extract an annulus 

In [ ]:
da.where(((da.Q>0.15) & (da.Q<0.35))).plot(norm=LogNorm(1e-16,1e-4))

or a sector

In [ ]:
da.where((np.abs(da.CHI)>75) & (np.abs(da.CHI)<105)).plot(norm=LogNorm(1e-16,1e-4))

and we can *almost* do the \chi vs q remeshing without pyFAI, but it doesn't come out quite right

In [ ]:
da.plot.pcolormesh(x='Q',y='CHI')

we can also use the multidimensional bins for groupby operations

In [ ]:
for i,sda in da.groupby_bins('Q',np.geomspace(1e-3,3.0,10),restore_coord_dims=True):
    fig,ax = plt.subplots()
    sda.unstack().sortby(['Qx','Qy']).plot(norm=LogNorm(1e-16,1e-4))
    ax.set(title=i)

In [ ]:
group = (da
         .groupby_bins('Q',np.geomspace(1e-3,3.0,10),restore_coord_dims=True)
         .apply(lambda x: x.groupby_bins('CHI',np.arange(-180,180,5)).mean())
        )
group.plot.line(x='CHI_bins')
plt.legend(bbox_to_anchor=(1.05,0.5),loc=6)